In [9]:
FILE = "/scratch/kshenoy/data/data/wikipedia_split/psgs_w100.tsv"
OPFILE = "/scratch/kshenoy/data/wikipedia_splits/psgs_w100.json"
OPDENSEFILE = "/scratch/kshenoy/data/dense_wikipedia_splits/dense_psgs_w100.json"

In [2]:
import json
from tqdm.notebook import tqdm

In [3]:
with open(FILE) as f:
    first_line = True
    cnt = 100
    with open(OPFILE, 'w') as fW:
        for line in tqdm(f, total=21015325):
            if first_line:
                first_line = False
                continue
            line = line.strip().split('\t')
            fW.write(json.dumps({"id": line[0], "contents": line[1], "title": line[2]}) + '\n')
#             if cnt < 0:
#                 break
#             cnt -= 1

  0%|          | 0/21015325 [00:00<?, ?it/s]

In [10]:
with open(FILE) as f:
    first_line = True
    cnt = 100
    with open(OPDENSEFILE, 'w') as fW:
        for line in tqdm(f, total=21015325):
            if first_line:
                first_line = False
                continue
            line = line.strip().split('\t')
            fW.write(json.dumps({"id": line[0], "contents": line[1]}) + '\n')
#             if cnt < 0:
#                 break
#             cnt -= 1

  0%|          | 0/21015325 [00:00<?, ?it/s]

In [3]:
import pyserini

In [6]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher('/scratch/kshenoy/data/indexes/psgs_w100')
hits = searcher.search('what race is aaron judge of the new york yankees')

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f} {json.loads(hits[i].raw)["contents"]}')

Exception: Unable to find javac

In [12]:
from collections import defaultdict
from pyserini.search import SimpleSearcher

top_K = 20
def fetch_all_retrieval_doc_ids(pred_file):
    predictions = defaultdict(list)
    with open(pred_file) as f:
        first_line = True
        for line in f:
            if first_line:
                first_line = False
                continue
            line = line.split()
            predictions[line[0]].append((line[2], line[4]))
    return predictions

def get_top_k_accuracy(top_k, predictions, searcher, target):
    hits = 0
    for docid in predictions.keys():
        doc = searcher.doc(docid)
        content = doc.contents()
        found = False
        for ans in target[int(docid)]['short_answers']:
            if ans in content:
                found = True
                break
        if found:
            hits += 1
    return 100 * hits / len(predictions.keys())

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--pred_file", type=str, help="predictions file")
    parser.add_argument("--index_file", type=str, help="index file")
    parser.add_argument("--target_file", type=str, help="target file")
    
    parser.add_argument("--top_k", type=int, default=20, help="top k retrieved results to be considered")
    
    args = parser.parse_args()
    
    PRED_FILE = args.pred_file # "/scratch/kshenoy/output/retrieval_results/gar_sample/run.sample.txt"
    INDEX_FILE = args.index_file # '/scratch/kshenoy/data/indexes/psgs_w100'
    TARGET_FILE = args.target_file # "/scratch/kshenoy/data/data/gold_passages_info/nq_test.json"

    target = json.load(TARGET_FILE)
    searcher = SimpleSearcher(INDEX_FILE)
    predictions = fetch_all_retrieval_doc_ids(PRED_FILE)
    
    print(get_top_k_accuracy(args.top_k, predictions, searcher, target))

Exception: Unable to find javac